In [ ]:
import os
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
class VideoObjectDetectionDataset(Dataset):
    def __init__(self, video_frames_path, mask_files_path, transform=None):
        self.video_frames_path = video_frames_path
        self.mask_files_path = mask_files_path
        self.transform = transform

        self.frame_files = sorted(os.listdir(video_frames_path))
        self.mask_files = sorted(os.listdir(mask_files_path))

    def __len__(self):
        return len(self.frame_files)

    def __getitem__(self, idx):
        frame_file = os.path.join(self.video_frames_path, self.frame_files[idx])
        mask_file = os.path.join(self.mask_files_path, self.mask_files[idx])

        frame = cv2.imread(frame_file)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
        mask = (mask > 128).astype(np.uint8)  # Convert the mask to binary

        sample = {'frame': frame, 'mask': mask}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
class ResizeAndNormalize(object):
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        frame, mask = sample['frame'], sample['mask']

        frame = cv2.resize(frame, self.output_size, interpolation=cv2.INTER_LINEAR)
        mask = cv2.resize(mask, self.output_size, interpolation=cv2.INTER_NEAREST)

        frame = frame / 255.0
        frame = frame.astype(np.float32)
        frame = np.transpose(frame, (2, 0, 1))

        return {'frame': frame, 'mask': mask}

In [ ]:
video_frames_path = "path/to/video/frames"
mask_files_path = "path/to/mask/files"

transform = ResizeAndNormalize(output_size=(224, 224))

dataset = VideoObjectDetectionDataset(video_frames_path, mask_files_path, transform=transform)

batch_size = 4
num_workers = 4

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
